**데이터 13만장 모두 이용하는 코드**

- 여기서 유효 데이터(횡단보도, 정지선이 있는 데이터)가 약 2만장

segmentation용 라벨을 detection용으로 변환하고 학습에 사용

# 필요한 라이브러리 설치 및 데이터셋 다운로드

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip -q install ultralytics

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 필요한 라이브러리 설치(캐글)
!pip uninstall -y kaggle kagglesdk -q
!pip install --upgrade pip
!pip install kaggle -q

In [ ]:
from google.colab import files
import os

# kaggle.json 파일 업로드
if not os.path.exists("/content/kaggle.json"):
    print("kaggle.json 파일을 업로드해주세요:")
    uploaded = files.upload()
    print("업로드 완료!")
else:
    print("kaggle.json 파일이 이미 존재합니다.")

kaggle.json 파일을 업로드해주세요:


Saving kaggle.json to kaggle.json
업로드 완료!


In [ ]:
# Kaggle API 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API 설정 완료!")

Kaggle API 설정 완료!


In [ ]:
!kaggle datasets download -d jeongbinhan/road-line-segmentation -p /content/traffic_sample --force
print("다운로드 완료!")

Dataset URL: https://www.kaggle.com/datasets/jeongbinhan/road-line-segmentation
License(s): unknown
100% 39.0G/39.0G [16:56<00:00, 41.2MB/s]

다운로드 완료!


In [ ]:
%cd /content/traffic_sample/
!unzip road-line-segmentation.zip
!rm road-line-segmentation.zip
!ls

Streaming output truncated to the last 5000 lines.
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062494.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062495.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062756.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062757.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062758.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062759.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062760.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062761.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062762.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_1280_720_daylight_train_8/7062763.json  
  inflating: 차선-횡단보도 인지 영상(수도권)/Training/label/c_

# 데이터 전처리



In [ ]:
%cd /content/traffic_sample/
os.rename('차선-횡단보도 인지 영상(수도권)', '123')

/content/traffic_sample


In [ ]:
# 전체 라벨 개수 확인
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_1')))
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_2')))
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_3')))
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_4')))
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_5')))
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_6')))
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_7')))
print(len(os.listdir('/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_8')))


30000
30000
30000
30000
30000
30000
30000
11081


In [ ]:
# 1번 데이터 압축풀기
%cd /content/traffic_sample/c_1280_720_daylight_train_1/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight
!cat c_1280_720_daylight_train_1.tar.part* > merged_train_1.tar
!tar -xvf merged_train_1.tar

# 2번 데이터 압축풀기
%cd /content/traffic_sample/c_1280_720_daylight_train_2/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight
!cat c_1280_720_daylight_train_2.tar.part* > merged_train_2.tar
!tar -xvf merged_train_2.tar

# 3번 데이터 압축풀기
%cd /content/traffic_sample/c_1280_720_daylight_train_3/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight
!cat c_1280_720_daylight_train_3.tar.part* > merged_train_3.tar
!tar xvf merged_train_3.tar

Streaming output truncated to the last 5000 lines.
c_1280_720_daylight_train_1/16925276.jpg
c_1280_720_daylight_train_1/16925277.jpg
c_1280_720_daylight_train_1/16925278.jpg
c_1280_720_daylight_train_1/16925279.jpg
c_1280_720_daylight_train_1/16925280.jpg
c_1280_720_daylight_train_1/16925281.jpg
c_1280_720_daylight_train_1/16925282.jpg
c_1280_720_daylight_train_1/16925283.jpg
c_1280_720_daylight_train_1/16925284.jpg
c_1280_720_daylight_train_1/16925285.jpg
c_1280_720_daylight_train_1/16925286.jpg
c_1280_720_daylight_train_1/16925287.jpg
c_1280_720_daylight_train_1/16925288.jpg
c_1280_720_daylight_train_1/16925289.jpg
c_1280_720_daylight_train_1/16925290.jpg
c_1280_720_daylight_train_1/16925291.jpg
c_1280_720_daylight_train_1/16925292.jpg
c_1280_720_daylight_train_1/16925293.jpg
c_1280_720_daylight_train_1/16925294.jpg
c_1280_720_daylight_train_1/16925295.jpg
c_1280_720_daylight_train_1/16925296.jpg
c_1280_720_daylight_train_1/16925297.jpg
c_1280_720_daylight_train_1/16925298.jpg
c_1280

In [ ]:
# 1~3번 데이터 개수 확인
print(len(os.listdir('/content/traffic_sample/c_1280_720_daylight_train_1/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight/c_1280_720_daylight_train_1')))
print(len(os.listdir('/content/traffic_sample/c_1280_720_daylight_train_2/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight/c_1280_720_daylight_train_2')))
print(len(os.listdir('/content/traffic_sample/c_1280_720_daylight_train_3/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight/c_1280_720_daylight_train_3')))

30000
30000
30000


In [ ]:
# 4, 5, 6, 7, 8번 데이터 압축 풀기
%cd /content/traffic_sample/123/Training/raw/
!tar -xvf c_1280_720_daylight_train_4.tar
!tar -xvf c_1280_720_daylight_train_5.tar
!tar -xvf c_1280_720_daylight_train_6.tar
!tar -xvf c_1280_720_daylight_train_7.tar
!tar -xvf c_1280_720_daylight_train_8.tar


Streaming output truncated to the last 5000 lines.
c_1280_720_daylight_train_8/7062485.jpg
c_1280_720_daylight_train_8/7062486.jpg
c_1280_720_daylight_train_8/7062487.jpg
c_1280_720_daylight_train_8/7062488.jpg
c_1280_720_daylight_train_8/7062490.jpg
c_1280_720_daylight_train_8/7062491.jpg
c_1280_720_daylight_train_8/7062492.jpg
c_1280_720_daylight_train_8/7062493.jpg
c_1280_720_daylight_train_8/7062494.jpg
c_1280_720_daylight_train_8/7062495.jpg
c_1280_720_daylight_train_8/7062756.jpg
c_1280_720_daylight_train_8/7062757.jpg
c_1280_720_daylight_train_8/7062758.jpg
c_1280_720_daylight_train_8/7062759.jpg
c_1280_720_daylight_train_8/7062760.jpg
c_1280_720_daylight_train_8/7062761.jpg
c_1280_720_daylight_train_8/7062762.jpg
c_1280_720_daylight_train_8/7062763.jpg
c_1280_720_daylight_train_8/7062764.jpg
c_1280_720_daylight_train_8/7062765.jpg
c_1280_720_daylight_train_8/7062766.jpg
c_1280_720_daylight_train_8/7062767.jpg
c_1280_720_daylight_train_8/7062768.jpg
c_1280_720_daylight_train_8/7

In [ ]:
# 4~8번 데이터 개수 확인
print(len(os.listdir('/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_4')))
print(len(os.listdir('/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_5')))
print(len(os.listdir('/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_6')))
print(len(os.listdir('/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_7')))
print(len(os.listdir('/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_8')))

30000
30000
30000
30000
11081


In [ ]:
# 데이터를 저장할 폴더 생성
%mkdir /content/data
%mkdir /content/data/images       # 원본 이미지
%mkdir /content/data/seg_labels   # 원본 라벨(polyline & polygon)

In [ ]:
# 이미지랑 라벨 각각 한 폴더에 모으기

import os
import shutil
from tqdm import tqdm

# ==========================================
# 1. 경로 설정
# ==========================================

# 이미지가 들어있는 소스 폴더들
image_source_dirs = [
    "/content/traffic_sample/c_1280_720_daylight_train_1/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight/c_1280_720_daylight_train_1",
    "/content/traffic_sample/c_1280_720_daylight_train_2/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight/c_1280_720_daylight_train_2",
    "/content/traffic_sample/c_1280_720_daylight_train_3/053.차선-횡단보도_인지_영상(수도권)/01.데이터/1.Training/원천데이터_0610/1280_720/daylight/c_1280_720_daylight_train_3",
    "/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_4",
    "/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_5",
    "/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_6",
    "/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_7",
    "/content/traffic_sample/123/Training/raw/c_1280_720_daylight_train_8",
]

# 라벨이 들어있는 소스 폴더들
label_source_dirs = [
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_1",
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_2",
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_3",
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_4",
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_5",
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_6",
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_7",
    "/content/traffic_sample/123/Training/label/c_1280_720_daylight_train_8",
]

# 목적지 폴더
target_image_dir = "/content/data/images/"
target_label_dir = "/content/data/seg_labels/"

# 목적지 폴더 생성
os.makedirs(target_image_dir, exist_ok=True)
os.makedirs(target_label_dir, exist_ok=True)

# ==========================================
# 2. 이동 함수 정의
# ==========================================
def move_all_files(source_dirs, target_dir, description):
    print(f"\n--- {description} 시작 ---")
    total_moved = 0

    for src_dir in source_dirs:
        if not os.path.exists(src_dir):
            print(f"경로 없음 (건너뜀): {src_dir}")
            continue

        files = os.listdir(src_dir)
        print(f"폴더 처리 중: {os.path.basename(src_dir)} (파일 수: {len(files)})")

        for f in tqdm(files, desc="이동 중"):
            src_path = os.path.join(src_dir, f)
            dst_path = os.path.join(target_dir, f)

            # 파일인 경우만 이동
            if os.path.isfile(src_path):
                shutil.move(src_path, dst_path)
                total_moved += 1

    print(f"--> {description} 완료! 총 {total_moved}개 파일 이동됨.")

# ==========================================
# 3. 실행
# ==========================================

# 이미지 이동 실행
move_all_files(image_source_dirs, target_image_dir, "이미지 이동")

# 라벨 이동 실행
move_all_files(label_source_dirs, target_label_dir, "라벨 이동")



--- 이미지 이동 시작 ---
폴더 처리 중: c_1280_720_daylight_train_1 (파일 수: 30000)


이동 중: 100%|██████████| 30000/30000 [00:00<00:00, 30808.63it/s]


폴더 처리 중: c_1280_720_daylight_train_2 (파일 수: 30000)


이동 중: 100%|██████████| 30000/30000 [00:01<00:00, 28308.40it/s]


폴더 처리 중: c_1280_720_daylight_train_3 (파일 수: 30000)


...

폴더 처리 중: c_1280_720_daylight_train_7 (파일 수: 30000)


이동 중: 100%|██████████| 30000/30000 [00:00<00:00, 30627.88it/s]


폴더 처리 중: c_1280_720_daylight_train_8 (파일 수: 11081)


이동 중: 100%|██████████| 11081/11081 [00:00<00:00, 30208.19it/s]

--> 라벨 이동 완료! 총 221081개 파일 이동됨.


In [ ]:
# 전체 데이터 : 약 22만장
print(f'images : {len(os.listdir('/content/data/images'))}')
print(f'labels : {len(os.listdir('/content/data/seg_labels'))}')

images : 221081
labels : 221081


# 라벨 변환(segmentation -> detection) 과정에서 padding 적용하기

In [ ]:
# 패딩을 적용한 라벨 변환 코드

import os
import glob
import json
import cv2
from tqdm import tqdm

# =========================
# 0) 경로 설정
# =========================
# JSON 라벨이 들어있는 폴더
JSON_DIR = "/content/data/seg_labels"

# 이미지들이 모여있는 폴더
IMG_DIR = "/content/data/images"

# 변환된 YOLO 라벨을 저장할 폴더
OUT_DET_DIR = "/content/data/det_labels_pad"

# =========================
# 1) 클래스 및 패딩 설정
# =========================
# 필요한 클래스만 남기고 ID 부여
KEEP_CLASSES = ["stop_line", "crosswalk"]
label2id = {c: i for i, c in enumerate(KEEP_CLASSES)}

# [수정됨] 박스 확장(Padding) 픽셀 수 (상하좌우 각각 적용)
BOX_PADDING = 10

# 저장 폴더 생성
os.makedirs(OUT_DET_DIR, exist_ok=True)

print(f"대상 클래스                  : {label2id}")
print(f"적용 패딩(px)                : {BOX_PADDING}")
print(f"이미지 경로                  : {IMG_DIR}")
print(f"라벨(json) 경로              : {JSON_DIR}")
print(f"detection 라벨(txt)저장 경로 : {OUT_DET_DIR}")

# =========================
# 2) 유틸 함수
# =========================
def get_img_wh(img_path):
    """이미지의 너비(w), 높이(h)를 반환"""
    im = cv2.imread(img_path)
    if im is None:
        return None, None
    h, w = im.shape[:2]
    return w, h

def points_to_xy(pts):
    """JSON의 좌표 데이터를 [(x,y), ...] 리스트로 변환"""
    if not isinstance(pts, list) or len(pts) == 0:
        return []
    # [{"x":1, "y":2}, ...] 형태
    if isinstance(pts[0], dict) and "x" in pts[0] and "y" in pts[0]:
        return [(float(p["x"]), float(p["y"])) for p in pts]
    # [[1, 2], ...] 형태
    if isinstance(pts[0], (list, tuple)) and len(pts[0]) == 2:
        return [(float(a), float(b)) for a, b in pts]
    return []

def xy_to_bbox(xy, w, h, padding=0):
    """
    다각형 좌표들로부터 Bounding Box (x1, y1, x2, y2) 추출
    [수정됨] padding 인자를 받아 박스를 확장하고 이미지 범위를 벗어나지 않게 처리
    """
    xs = [p[0] for p in xy]
    ys = [p[1] for p in xy]

    # 1. Min/Max 좌표 구하기 (패딩 적용)
    min_x = min(xs) - padding
    max_x = max(xs) + padding
    min_y = min(ys) - padding
    max_y = max(ys) + padding

    # 2. 이미지 범위를 벗어나지 않도록 클램핑(clamp)
    # 0보다 작아지면 0으로, w-1보다 커지면 w-1로 고정
    x1 = max(0.0, min_x)
    x2 = min(float(w-1), max_x)
    y1 = max(0.0, min_y)
    y2 = min(float(h-1), max_y)

    # 3. 유효하지 않은 박스 제외 (너비나 높이가 0 이하인 경우)
    if x2 <= x1 or y2 <= y1:
        return None
    return x1, y1, x2, y2

def bbox_to_yolo(x1, y1, x2, y2, w, h):
    """Bounding Box를 YOLO 형식 (center_x, center_y, w, h) 정규화 좌표로 변환"""
    xc = ((x1 + x2) / 2.0) / w
    yc = ((y1 + y2) / 2.0) / h
    bw = (x2 - x1) / w
    bh = (y2 - y1) / h

    # 값 범위를 0~1 사이로 강제
    xc = max(0.0, min(1.0, xc))
    yc = max(0.0, min(1.0, yc))
    bw = max(0.0, min(1.0, bw))
    bh = max(0.0, min(1.0, bh))
    return xc, yc, bw, bh

# =========================
# 3) 사전 작업: 이미지 파일 매핑 (속도 최적화)
# =========================
print("이미지 목록 스캔 중...")
image_map = {}
valid_exts = {".jpg", ".png", ".jpeg", ".JPG", ".PNG", ".JPEG"}

for f in tqdm(os.listdir(IMG_DIR), desc="Indexing Images"):
    name, ext = os.path.splitext(f)
    if ext in valid_exts:
        image_map[name] = os.path.join(IMG_DIR, f)

print(f"총 {len(image_map)}개의 이미지를 찾았습니다.")

# =========================
# 4) JSON 파일 목록 로드
# =========================
json_paths = glob.glob(os.path.join(JSON_DIR, "*.json"))
print(f"총 {len(json_paths)}개의 JSON 파일을 찾았습니다. 변환을 시작합니다.")

# =========================
# 5) 변환 루프 실행
# =========================
converted_count = 0
skipped_noimg = 0
skipped_empty = 0

for jp in tqdm(json_paths, desc="Converting JSON to YOLO"):
    stem = os.path.splitext(os.path.basename(jp))[0]

    img_path = image_map.get(stem)
    if img_path is None:
        skipped_noimg += 1
        continue

    w, h = get_img_wh(img_path)
    if w is None:
        skipped_noimg += 1
        continue

    try:
        with open(jp, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        print(f"JSON Error ({stem}): {e}")
        continue

    anns = data.get("annotations", [])
    if not isinstance(anns, list) or len(anns) == 0:
        skipped_empty += 1
        continue

    yolo_lines = []

    for ann in anns:
        cls = ann.get("class", None)
        cat = ann.get("category", None)

        if cls not in label2id:
            continue
        if cat not in ["polyline", "polygon"]:
            continue

        pts = ann.get("data", [])
        xy = points_to_xy(pts)

        if cat == "polyline" and len(xy) < 2: continue
        if cat == "polygon" and len(xy) < 3: continue

        # [수정됨] padding 값을 인자로 전달
        bbox = xy_to_bbox(xy, w, h, padding=BOX_PADDING)
        if bbox is None:
            continue

        x1, y1, x2, y2 = bbox
        xc, yc, bw, bh = bbox_to_yolo(x1, y1, x2, y2, w, h)

        class_id = label2id[cls]
        yolo_lines.append(f"{class_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

    if len(yolo_lines) > 0:
        out_txt_path = os.path.join(OUT_DET_DIR, stem + ".txt")
        with open(out_txt_path, "w", encoding="utf-8") as f:
            f.write("\n".join(yolo_lines) + "\n")
        converted_count += 1
    else:
        skipped_empty += 1

# =========================
# 6) 결과 리포트
# =========================
print("\n" + "="*40)
print(f"변환 완료: {converted_count}개 파일 생성됨")
print(f"적용된 패딩: {BOX_PADDING} px")
print(f"이미지 없음(건너뜀): {skipped_noimg}개")
print(f"유효 라벨 없음(건너뜀): {skipped_empty}개")
print(f"결과 저장 경로: {OUT_DET_DIR}")
print("="*40)

대상 클래스                  : {'stop_line': 0, 'crosswalk': 1}
적용 패딩(px)                : 10
이미지 경로                  : /content/data/images
라벨(json) 경로              : /content/data/seg_labels
detection 라벨(txt)저장 경로 : /content/data/det_labels_pad
이미지 목록 스캔 중...


Indexing Images: 100%|██████████| 221081/221081 [00:00<00:00, 251813.40it/s]


총 221081개의 이미지를 찾았습니다.
총 221081개의 JSON 파일을 찾았습니다. 변환을 시작합니다.


Converting JSON to YOLO: 100%|██████████| 221081/221081 [34:56<00:00, 105.44it/s]


변환 완료: 32874개 파일 생성됨
적용된 패딩: 10 px
이미지 없음(건너뜀): 0개
유효 라벨 없음(건너뜀): 188207개
결과 저장 경로: /content/data/det_labels_pad


In [ ]:
# 변환된 라벨 개수 확인
print(f'images : {len(os.listdir('/content/data/images'))}')
print(f'json labels : {len(os.listdir('/content/data/seg_labels'))}')
print(f'txt labels(padded) : {len(os.listdir('/content/data/det_labels_pad'))}')

images : 221081
json labels : 221081
txt labels(padded) : 32874


In [ ]:
# 변환된 라벨 시각화해보기

import os
import random
import cv2
import matplotlib.pyplot as plt

# =========================
# 1. 설정 (경로 및 클래스)
# =========================
IMG_DIR = "/content/data/images"       # 이미지가 있는 폴더
LABEL_DIR = "/content/data/det_labels_pad"     # 변환된 YOLO 라벨 폴더

# 클래스 정의 (ID -> 이름)
CLASS_MAP = {
    0: 'stopline',
    1: 'crosswalk'
}

# 시각화 색상 (R, G, B) - 정지선: 빨강, 횡단보도: 초록
COLORS = {
    0: (255, 0, 0),   # Red
    1: (0, 255, 0)    # Green
}

def visualize_random_sample():
    # 1. 라벨 파일 목록 가져오기
    label_files = [f for f in os.listdir(LABEL_DIR) if f.endswith('.txt')]

    if not label_files:
        print("라벨 파일이 없습니다. 경로를 확인해주세요.")
        return

    # 2. 랜덤하게 하나 선택
    sample_label = random.choice(label_files)
    stem = os.path.splitext(sample_label)[0]

    # 3. 대응되는 이미지 찾기
    # (jpg, png 등 확장자 자동 탐색)
    img_path = None
    for ext in ['.jpg', '.png', '.jpeg', '.JPG']:
        temp_path = os.path.join(IMG_DIR, stem + ext)
        if os.path.exists(temp_path):
            img_path = temp_path
            break

    if img_path is None:
        print(f"이미지 파일을 찾을 수 없습니다: {stem}")
        return

    # 4. 이미지 로드
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB 변환
    h, w, _ = img.shape

    # 5. 라벨 읽기 및 그리기
    label_path = os.path.join(LABEL_DIR, sample_label)
    with open(label_path, 'r') as f:
        lines = f.readlines()

    print(f"파일명: {sample_label} | 이미지 크기: {w}x{h}")

    for line in lines:
        parts = list(map(float, line.split()))
        class_id = int(parts[0])
        x_center, y_center, bbox_w, bbox_h = parts[1], parts[2], parts[3], parts[4]

        # --- 좌표 복원 (De-normalization) ---
        # 중심 좌표와 너비/높이를 픽셀 단위로 변환
        x_c_px = x_center * w
        y_c_px = y_center * h
        w_px = bbox_w * w
        h_px = bbox_h * h

        # 좌상단(x1, y1), 우하단(x2, y2) 좌표 계산
        x1 = int(x_c_px - (w_px / 2))
        y1 = int(y_c_px - (h_px / 2))
        x2 = int(x_c_px + (w_px / 2))
        y2 = int(y_c_px + (h_px / 2))

        # --- 그리기 ---
        color = COLORS.get(class_id, (255, 255, 0)) # 지정 안 된 클래스는 노란색
        label_name = CLASS_MAP.get(class_id, str(class_id))

        # 박스 그리기
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

        # 텍스트 배경 및 텍스트
        text_size = cv2.getTextSize(label_name, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
        cv2.rectangle(img, (x1, y1 - text_size[1] - 10), (x1 + text_size[0], y1), color, -1)
        cv2.putText(img, label_name, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # 6. 출력
    plt.figure(figsize=(15, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Sample: {stem}", fontsize=15)
    plt.show()

# 실행 (여러 번 실행하면 다른 이미지가 나옵니다)
visualize_random_sample()

In [19]:
# 한 번에 여러 이미지 시각화
import os
import random
import cv2
import matplotlib.pyplot as plt

# =========================
# 1. 설정 (경로 및 클래스)
# =========================
IMG_DIR = "/content/data/images"               # 이미지가 있는 폴더
LABEL_DIR = "/content/data/det_labels_pad"     # 변환된 YOLO 라벨 폴더

# 클래스 정의 (ID -> 이름)
CLASS_MAP = {
    0: 'stopline',
    1: 'crosswalk'
}

# 시각화 색상 (R, G, B)
COLORS = {
    0: (255, 0, 0),   # Red
    1: (0, 255, 0)    # Green
}

def visualize_9_samples():
    # 1. 라벨 파일 목록 가져오기
    label_files = [f for f in os.listdir(LABEL_DIR) if f.endswith('.txt')]

    if not label_files:
        print("라벨 파일이 없습니다. 경로를 확인해주세요.")
        return

    # 2. 랜덤하게 9개 선택 (파일이 9개보다 적으면 전체 선택)
    num_samples = min(9, len(label_files))
    selected_files = random.sample(label_files, num_samples)

    # 3. 전체 그래프 크기 설정 (가로, 세로)
    plt.figure(figsize=(20, 14))

    # 4. 선택된 파일들에 대해 반복 처리
    for i, sample_label in enumerate(selected_files):
        stem = os.path.splitext(sample_label)[0]

        # --- 이미지 찾기 ---
        img_path = None
        for ext in ['.jpg', '.png', '.jpeg', '.JPG']:
            temp_path = os.path.join(IMG_DIR, stem + ext)
            if os.path.exists(temp_path):
                img_path = temp_path
                break

        if img_path is None:
            print(f"이미지 없음: {stem}")
            continue

        # --- 이미지 로드 ---
        img = cv2.imread(img_path)
        if img is None:
            print(f"이미지 로드 실패: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # --- 라벨 읽기 및 그리기 ---
        label_path = os.path.join(LABEL_DIR, sample_label)
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = list(map(float, line.split()))
            class_id = int(parts[0])
            x_center, y_center, bbox_w, bbox_h = parts[1], parts[2], parts[3], parts[4]

            # 좌표 복원
            x_c_px = x_center * w
            y_c_px = y_center * h
            w_px = bbox_w * w
            h_px = bbox_h * h

            x1 = int(x_c_px - (w_px / 2))
            y1 = int(y_c_px - (h_px / 2))
            x2 = int(x_c_px + (w_px / 2))
            y2 = int(y_c_px + (h_px / 2))

            # 그리기
            color = COLORS.get(class_id, (255, 255, 0))
            label_name = CLASS_MAP.get(class_id, str(class_id))

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

            # 텍스트 크기 계산 (이미지가 작게 보일 수 있으므로 폰트 크기 약간 조절)
            text_size = cv2.getTextSize(label_name, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
            cv2.rectangle(img, (x1, y1 - text_size[1] - 10), (x1 + text_size[0], y1), color, -1)
            cv2.putText(img, label_name, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # --- 서브플롯 추가 ---
        # 3행 3열 중 i+1 번째 위치에 그림
        plt.subplot(3, 3, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{stem}", fontsize=12)

    # 5. 전체 출력
    plt.tight_layout()
    plt.show()

# 실행
visualize_9_samples()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 전체 데이터 중 학습에 사용할(유효 라벨이 있는) 3만장만 옮기기
# ./data/images 폴더에는 이미지 22만장 있음
# 이 중 라벨이 있는 3만장을 ./data/det_images/로 옮길거

import os
import shutil
from tqdm import tqdm

%mkdir /content/data/det_images/

# 경로 설정 (실제 경로에 맞게 수정하세요)
images_dir = '/content/data/images/'
labels_dir = '/content/data/det_labels_pad/'
target_dir = '/content/data/det_images/'


# 1. 라벨 폴더 내의 파일명(확장자 제외)을 집합(Set)으로 저장
# 집합을 사용하면 검색 속도가 O(1)로 매우 빠릅니다.
label_filenames = {os.path.splitext(f)[0] for f in os.listdir(labels_dir) if f.endswith('.txt')}

print(f"찾은 라벨 개수: {len(label_filenames)}개")

# 2. 이미지 폴더 내 파일을 확인하며 이동
moved_count = 0
for img_file in tqdm(os.listdir(images_dir), desc="이미지 선별 중"):
    name, ext = os.path.splitext(img_file)

    # 확장자가 이미지인 것 중, 라벨 이름 집합에 포함된 경우만 이동
    if ext.lower() in ['.jpg', '.jpeg', '.png'] and name in label_filenames:
        src_path = os.path.join(images_dir, img_file)
        dst_path = os.path.join(target_dir, img_file)

        # 파일 복사(copy) 또는 이동(move) 중 선택
        shutil.copy(src_path, dst_path)
        moved_count += 1

print(f"\n작업 완료! 총 {moved_count}개의 이미지를 {target_dir}로 옮겼습니다.")

찾은 라벨 개수: 32874개


이미지 선별 중: 100%|██████████| 221081/221081 [02:20<00:00, 1570.21it/s]


작업 완료! 총 32874개의 이미지를 /content/data/det_images/로 옮겼습니다.


In [5]:
print(len(os.listdir('/content/data/images')))          # 13만장
print(len(os.listdir('/content/data/det_images')))      # 3만장            -> 이거랑
print(len(os.listdir('/content/data/seg_labels')))      # 13만개
print(len(os.listdir('/content/data/det_labels_pad')))  # 3만개 인지 확인  -> 이거를 학습에 사용

221081
32874
221081
32874


In [ ]:
# 유효한 라벨이 있는 샘플 목록 만들기
import os

%cd /content/data

# 1. 경로 설정
TARGET_DIR = "/content/data/det_labels_pad"  # 파일들이 있는 폴더 경로
OUTPUT_FILE = "valid_label_list.txt"              # 저장할 파일 이름

# 2. 파일 목록 가져오기
file_names = os.listdir(TARGET_DIR)

# 3. 정렬 (선택 사항)
# 그냥 정렬하면 1.txt -> 10.txt -> 2.txt 순서가 됩니다.
# 숫자 순서대로(1 -> 2 -> 10) 정렬하려면 아래 코드를 사용하세요.
try:
    file_names.sort(key=lambda x: int(os.path.splitext(x)[0]))
except ValueError:
    # 파일명에 숫자가 아닌 것이 섞여있다면 그냥 이름순 정렬
    file_names.sort()

# 4. 파일에 쓰기
with open(OUTPUT_FILE, "w") as f:
    for name in file_names:
        f.write(os.path.splitext(name)[0] + "\n")

print(f"완료! 총 {len(file_names)}개의 파일명이 '{OUTPUT_FILE}'에 저장되었습니다.")

/content/data
완료! 총 32874개의 파일명이 'valid_label_list.txt'에 저장되었습니다.


# 데이터 train-val-test로 쪼개기


In [ ]:
import os
import glob
import shutil
import random
from pathlib import Path

# --- 설정값 ---
# 기존 데이터 경로
src_img_path = "/content/data/det_images"
src_label_path = "/content/data/det_labels_pad"

# 데이터가 이동될 최종 경로 (YOLO 표준 구조)
dest_root = "/content/datasets"

# 분할 비율 (Train : Val : Test) = 8 : 1 : 1
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# 랜덤 시드 설정
random_seed = 42

# ---------------------------------------------

def split_dataset():
    # 1. 파일 리스트 확보 (라벨 파일 기준)
    # 이미지 확장자는 jpg라고 가정하지만, 필요시 수정 가능
    label_files = glob.glob(os.path.join(src_label_path, "*.txt"))

    # 짝이 맞는 데이터만 찾기 위해 파일명(stem) 추출
    # 예: "img_01.txt" -> "img_01"
    data_pairs = []

    print(f"원본 라벨 파일 검색 중... 총 {len(label_files)}개 발견")

    for label_file in label_files:
        basename = os.path.splitext(os.path.basename(label_file))[0]

        # 대응되는 이미지 파일 찾기 (jpg, png 등 확인)
        # 우선순위: jpg -> png -> jpeg
        image_file = os.path.join(src_img_path, f"{basename}.jpg")

        if not os.path.exists(image_file):
            # 혹시 jpg가 아니라면 다른 확장자 체크
            for ext in ['.png', '.jpeg', '.JPG']:
                temp_path = os.path.join(src_img_path, f"{basename}{ext}")
                if os.path.exists(temp_path):
                    image_file = temp_path
                    break

        if os.path.exists(image_file):
            data_pairs.append((image_file, label_file))
        else:
            print(f"[경고] 이미지 파일을 찾을 수 없음: {basename}")

    total_count = len(data_pairs)
    print(f"유효한 이미지-라벨 쌍: {total_count}개")

    # 2. 데이터 섞기 (Shuffle)
    random.seed(random_seed)
    random.shuffle(data_pairs)

    # 3. 분할 인덱스 계산
    train_end = int(total_count * train_ratio)
    val_end = train_end + int(total_count * val_ratio)

    train_set = data_pairs[:train_end]
    val_set = data_pairs[train_end:val_end]
    test_set = data_pairs[val_end:]

    print(f"\n[분할 결과]")
    print(f"- Train: {len(train_set)}개")
    print(f"- Val  : {len(val_set)}개")
    print(f"- Test : {len(test_set)}개")

    # 4. 폴더 생성 및 파일 이동 함수
    def move_files(file_list, split_name):
        # 목표 디렉토리 생성
        img_dest = os.path.join(dest_root, "images", split_name)
        lbl_dest = os.path.join(dest_root, "labels", split_name)

        os.makedirs(img_dest, exist_ok=True)
        os.makedirs(lbl_dest, exist_ok=True)

        for img, lbl in file_list:
            # shutil.move: 파일 이동 (원본 삭제됨)
            # 원본 보존을 원하면 shutil.copy로 변경하세요
            shutil.copy(img, os.path.join(img_dest, os.path.basename(img)))
            shutil.copy(lbl, os.path.join(lbl_dest, os.path.basename(lbl)))

    # 5. 실제 이동 실행
    print("\n파일 이동을 시작합니다...")
    move_files(train_set, "train")
    move_files(val_set, "val")
    move_files(test_set, "test")

    print("모든 작업이 완료되었습니다! '/content/datasets/' 경로를 확인하세요.")

# 실행
split_dataset()

원본 라벨 파일 검색 중... 총 32874개 발견
유효한 이미지-라벨 쌍: 32874개

[분할 결과]
- Train: 26299개
- Val  : 3287개
- Test : 3288개

파일 이동을 시작합니다...
모든 작업이 완료되었습니다! '/content/datasets/' 경로를 확인하세요.


In [6]:
# 분할된 데이터 개수 확인
print(f'train images : {len(os.listdir('/content/datasets/images/train/'))}')
print(f'train labels : {len(os.listdir('/content/datasets/labels/train/'))}')
print(f'val images : {len(os.listdir('/content/datasets/images/val/'))}')
print(f'val labels : {len(os.listdir('/content/datasets/labels/val/'))}')
print(f'test images : {len(os.listdir('/content/datasets/images/test/'))}')
print(f'test labels : {len(os.listdir('/content/datasets/labels/test/'))}')

train images : 26299
train labels : 26299
val images : 3287
val labels : 3287
test images : 3288
test labels : 3288


# 유효 라벨만 가지고 모델 학습

In [7]:
import yaml

# YAML에 들어갈 내용 설정
yaml_data = {
    'path': '/content/datasets', # 데이터셋 루트 경로
    'train': 'images/train',      # 학습 이미지 경로
    'val': 'images/val',        # 검증 이미지 경로
    'test': 'images/test',       # 테스트 이미지 경로

    'nc': 2,                 # 클래스 개수
    'names': [
        'stopline',
        'crosswalk'
        ] # 클래스 이름 (인덱스 순서대로)
}

# yaml 파일 저장
yaml_path = '/content/datasets/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_data, f, default_flow_style=False)

print(f"✅ {yaml_path} 생성이 완료되었습니다!")

# 생성된 내용 확인
with open(yaml_path, 'r') as f:
    print("\n--- data.yaml 내용 ---")
    print(f.read())

✅ /content/datasets/data.yaml 생성이 완료되었습니다!

--- data.yaml 내용 ---
names:
- stopline
- crosswalk
nc: 2
path: /content/datasets
test: images/test
train: images/train
val: images/val



In [13]:
from ultralytics import YOLO

%cd /content/

# 1. 모델 불러오기
model_name = "/content/padding_cross_stop.pt"

max_epochs = 50
imgsize    = 960
batch_size = 24

#save_dir = f"stopline_det_{model_name.split('.')[0]}_e{max_epochs}_b{batch_size}_imgsz{imgsize}"
save_dir = "stopline_det_large_data_e50_b24_img960"

model = YOLO(model_name)

train_results = model.train(
    data = "/content/datasets/data.yaml",  # Path to dataset configuration file
    epochs = max_epochs,  # Number of training epochs
    imgsz = imgsize,  # Image size for training
    batch = batch_size,
    device = 0,  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
    name = save_dir,
    val =True,
    plots = True,
    # 추가 하이퍼파라미터 튜닝
    patience = 0,               # early stopping 사용 안함
    dfl = 2.0,                  # Distributional Focal Loss 가중치 키움
    box = 6.0,                  # Bounding box loss 가중치 줄임
    copy_paste = 0.2            # Data Augmentation 기법 적용
)

/content
Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26x.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=stopline_det_yolo26x_e50_b6_imgsz9604, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_ma

KeyboardInterrupt: 

In [ ]:
# 구글드라이브에 학습 결과 저장

from google.colab import drive
import os
import datetime
import subprocess

# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

# ==========================================
# 설정
# ==========================================

dd = datetime.datetime.now().strftime("%y%m%d")
source_folder = f'/content/runs/detect/{save_dir}'          # 복사할 원본 폴더
dest_folder = f'/content/drive/MyDrive/det_{dd}_yolo26x'
config_folder = os.path.join(dest_folder, save_dir)         # 실험 설정 기록 - 폴더 이름으로

# ==========================================
# 실행 로직
# ==========================================
# 저장할 폴더 생성
os.makedirs(dest_folder, exist_ok=True)
os.makedirs(config_folder, exist_ok=True)

print(f"📂 '{source_folder}'의 내용을 '{dest_folder}'로 복사합니다...")

# rsync 명령어 실행 (-a: 아카이브 모드, -v: 상세출력, --progress: 진행률)
# 폴더 통째로 복사
command = f"rsync -av --progress '{source_folder}/' '{dest_folder}'"

# 쉘 명령어 실행
subprocess.call(command, shell=True)

print("\n✅ 복사 작업이 완료되었습니다!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 '/content/runs/detect/stopline_det_yolo26x_e50_b24_imgsz960'의 내용을 '/content/drive/MyDrive/det_260203_yolo26x'로 복사합니다...

✅ 복사 작업이 완료되었습니다!


In [ ]:
# 테스트 셋에서 모델 성능 측정

from ultralytics import YOLO

# 1. 모델 로드 및 검증 수행
model = YOLO(f'/content/runs/detect/{save_dir}/weights/best.pt')  # 경로 확인 필요
metrics = model.val(data='/content/datasets/data.yaml', split='test')

# 2. 전체적인 성능 지표 출력
print(f"mAP50-95: {metrics.box.map}")    # 전반적인 성능 (IoU 0.5~0.95 평균)
print(f"mAP50: {metrics.box.map50}")      # IoU 0.5 기준 성능 (일반적으로 많이 보는 지표)
print(f"mAP75: {metrics.box.map75}")      # IoU 0.75 기준 (더 엄격한 기준)

# 3. 클래스별 성능 확인 (신호등 종류가 여러 개인 경우 유용)
# metrics.box.maps는 각 클래스별 mAP50-95 값을 배열로 가지고 있습니다.
print(f"Class indices: {metrics.box.maps}")

Ultralytics 8.4.10 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLO26x summary (fused): 190 layers, 55,635,858 parameters, 0 gradients, 193.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2463.9±673.1 MB/s, size: 220.8 KB)
val: Scanning /content/datasets/labels/test... 2072 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2072/2072 1.5Kit/s 1.4s
val: New cache created: /content/datasets/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 130/130 5.9it/s 22.2s
                   all       2072       4303      0.838      0.708      0.772      0.528
              stopline       1363       1626      0.801      0.545      0.631      0.389
             crosswalk       1780       2677      0.875      0.871      0.913      0.667
Speed: 0.9ms preprocess, 6.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /content/runs/detect/val
mAP50-95: 0.5280727847965